### Generate the csvs for all splits

In [2]:
import torch
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments
import wandb 
import csv
import json
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
# from transformers import DistilBertTokenizer
import numpy as np
import torch.nn as nn
# import torch.optim as optim
# from tabulate import tabulate
# from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import torch

In [1]:
from loader import ContractNLIExample

ENTAILMENT = 0
CONTRADICTION = 1
NOT_MENTIONED = 2

def get_label(label):
    label = str(label)
    if label == "NLILabel.ENTAILMENT":
        return ENTAILMENT
    elif label == "NLILabel.CONTRADICTION":
        return CONTRADICTION
    elif label == "NLILabel.NOT_MENTIONED":
        return NOT_MENTIONED

In [3]:
# make a collected dataset
def json_to_csv(json_file, csv_file):
    # Read the JSON file
    examples = ContractNLIExample.load(json.load(open(json_file, 'r', encoding='utf-8')))

    with open(csv_file, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['concatenated_spans', 'hypothesis', 'num_spans', 'num_evidence_spans', 'num_tokens', 'label'])   # Write the header row

        # Iterate over each document
        for example in examples:
            doc_id = example.document_id  # Get the document ID
            spans = example.spans  # Get the 'spans' array
            num_spans = len(spans)  # Get the number of spans
            num_evidence_spans = len(example.annotated_spans)
            num_tokens = len(example.tokens)
            label = get_label(example.label)     # Get the 'label' object
            hypothesis = example.hypothesis_text  # Get the 'hypothesis' object
            annotated_spans = [spans[i] for i in example.annotated_spans]  # Get the 'annotations' object
            
            # check if label is 'not mentioned'
            if len(annotated_spans) == 0:
                # pick 3 random spans
                annotated_spans = [spans[i] for i in np.random.choice(len(spans), 3, replace=False)]

            context_text = example.context_text  # Get the 'context' object

            concatenated_spans = []
            concatenated_spans.append(
                "\n".join([context_text[start:end] for start, end in annotated_spans])
            )

            concatenated_spans = "\n\n".join(concatenated_spans)

            writer.writerow([concatenated_spans, hypothesis, num_spans, num_evidence_spans, num_tokens, label])  # Write the row to the CSV file

    print(f"Conversion complete. CSV file '{csv_file}' created.")

In [4]:
json_folder = "../dataset/contract-nli"
csv_folder = "../dataset/csv/all_labels"

In [5]:
# json_to_csv(f'{json_folder}/train.json', f'{csv_folder}/train.csv')
# json_to_csv(f'{json_folder}/dev.json', f'{csv_folder}/validation.csv')
# json_to_csv(f'{json_folder}/test.json', f'{csv_folder}/test.csv')

train_df = pd.read_csv(f'{csv_folder}/train.csv')
val_df = pd.read_csv(f'{csv_folder}/validation.csv')
test_df = pd.read_csv(f'{csv_folder}/test.csv')

100%|██████████| 423/423 [00:41<00:00, 10.08it/s]


Conversion complete. CSV file '../dataset/csv/all_labels/train.csv' created.


100%|██████████| 61/61 [00:06<00:00,  9.89it/s]


Conversion complete. CSV file '../dataset/csv/all_labels/validation.csv' created.


100%|██████████| 123/123 [00:11<00:00, 10.62it/s]


Conversion complete. CSV file '../dataset/csv/all_labels/test.csv' created.
